In [30]:
%%bash
python evaluate/eval_semantic_instance.py

OpenScene + Mask3D + MaPLe
torch.Size([423007, 94]) torch.Size([94]) (94,)
evaluating 1 scans...
(94,)
torch.Size([94])
torch.Size([423007, 94])
scans processed: 1

################################################################
what           :             AP         AP_50%         AP_25%
################################################################
basket         :            nan            nan            nan
bed            :            nan            nan            nan
bench          :            nan            nan            nan
bin            :          0.396          0.583          0.583
blanket        :          0.243          0.332          0.458
blinds         :          0.000          0.000          0.000
book           :            nan            nan            nan
bottle         :            nan            nan            nan
box            :            nan            nan            nan
bowl           :            nan            nan            nan
camera         :       

## generate label with OpenScene and ground truth masks

In [4]:
import experiment
current_path = experiment.get_current_path()
current_path

'/cluster/54/nanriayaka/ml3d/experiments/merged_pipline/run_2025-01-28-04-03-04'

In [5]:
from clip_utils import extract_text_feature, REPLICA_LABELS#, MATTERPORT_LABELS_21
labelset = list(REPLICA_LABELS)
text_features, new_label_set = extract_text_feature(labelset)
labelset.append('unlabeled')
labelset, new_label_set

Use prompt engineering: a XX in a scene
Loading CLIP ViT-B/32 model...
Finish loading


(['basket',
  'bed',
  'bench',
  'bin',
  'blanket',
  'blinds',
  'book',
  'bottle',
  'box',
  'bowl',
  'camera',
  'cabinet',
  'candle',
  'chair',
  'clock',
  'cloth',
  'comforter',
  'cushion',
  'desk',
  'desk-organizer',
  'door',
  'indoor-plant',
  'lamp',
  'monitor',
  'nightstand',
  'panel',
  'picture',
  'pillar',
  'pillow',
  'pipe',
  'plant-stand',
  'plate',
  'pot',
  'sculpture',
  'shelf',
  'sofa',
  'stool',
  'switch',
  'table',
  'tablet',
  'tissue-paper',
  'tv-screen',
  'tv-stand',
  'vase',
  'vent',
  'wall-plug',
  'window',
  'rug',
  'unlabeled'],
 ['a photo of a basket.',
  'a photo of a bed.',
  'a photo of a bench.',
  'a photo of a bin.',
  'a photo of a blanket.',
  'a photo of a blinds.',
  'a photo of a book.',
  'a photo of a bottle.',
  'a photo of a box.',
  'a photo of a bowl.',
  'a photo of a camera.',
  'a photo of a cabinet.',
  'a photo of a candle.',
  'a photo of a chair.',
  'a photo of a clock.',
  'a photo of a cloth.',
 

In [7]:
import os
gt_instance_path = os.path.join("dataset", "data", "replica_features")
# instance_path = os.path.join(current_path, "instance_features")
npy_files = [
    os.path.join(root, file)
    for root, _, files in os.walk(gt_instance_path)
    for file in files
    if file.endswith("_features.npy")
]
npy_files

['dataset/data/replica_features/test/office4_features.npy',
 'dataset/data/replica_features/test/room2_features.npy',
 'dataset/data/replica_features/val/room1_features.npy',
 'dataset/data/replica_features/val/office3_features.npy',
 'dataset/data/replica_features/train/office1_features.npy',
 'dataset/data/replica_features/train/room0_features.npy',
 'dataset/data/replica_features/train/office0_features.npy',
 'dataset/data/replica_features/train/office2_features.npy']

In [8]:
import numpy as np
from clip_utils import classify_features
import torch
os.makedirs(os.path.join(current_path, "instance_features_with_gt"), exist_ok=True)
for file in npy_files:
    instance_features = np.load(file)
    instance_features = torch.Tensor(instance_features)
    
    sample_name = os.path.basename(file).split('_')[0]
    
    print(f"Processing {sample_name}")
    
    print(text_features.shape)
    print(instance_features.shape)
    
    predicted_classes, confidence_scores = classify_features(text_features, instance_features)

    # save_path = os.path.dirname(file)
    save_path = os.path.join(current_path, "instance_features_with_gt")
    torch.save(predicted_classes, os.path.join(save_path, f"{sample_name}_predicted_classes.pl"))
    torch.save(confidence_scores, os.path.join(save_path, f"{sample_name}_confidence_scores.pl"))

Processing office4
torch.Size([48, 512])
torch.Size([71, 512])
Processing room2
torch.Size([48, 512])
torch.Size([61, 512])
Processing room1
torch.Size([48, 512])
torch.Size([57, 512])
Processing office3
torch.Size([48, 512])
torch.Size([113, 512])
Processing office1
torch.Size([48, 512])
torch.Size([52, 512])
Processing room0
torch.Size([48, 512])
torch.Size([94, 512])
Processing office0
torch.Size([48, 512])
torch.Size([68, 512])
Processing office2
torch.Size([48, 512])
torch.Size([94, 512])


In [8]:
torch.load(os.path.join(save_path, "room2_confidence_scores.pl"))

tensor([0.0224, 0.0224, 0.0225, 0.0224, 0.0224, 0.0224, 0.0224, 0.0225, 0.0232,
        0.0221, 0.0220, 0.0221, 0.0220, 0.0221, 0.0224, 0.0221, 0.0233, 0.0227,
        0.0230, 0.0234, 0.0227, 0.0232, 0.0232, 0.0236, 0.0224, 0.0224, 0.0219,
        0.0222, 0.0218, 0.0224, 0.0224, 0.0224, 0.0224, 0.0232, 0.0225, 0.0223,
        0.0220, 0.0222, 0.0224, 0.0235, 0.0227, 0.0221, 0.0222, 0.0219, 0.0221,
        0.0224, 0.0224, 0.0224, 0.0224, 0.0224, 0.0224, 0.0224, 0.0224, 0.0224,
        0.0224, 0.0221, 0.0225, 0.0226, 0.0223, 0.0224, 0.0223])